In [15]:
import urllib.request, json, datetime, time,pytz
import requests
from requests.structures import CaseInsensitiveDict
with open('keys.json') as json_file:
    keys = json.load(json_file)
trainkey = keys["trainkey"]
buskey = keys["buskey"]

#CTA bus and train trackers require an API key from the CTA.
#the following links have how to get an API key for bus and train trackers:
#https://www.transitchicago.com/developers/bustracker/
#https://www.transitchicago.com/developers/ttdocs/

#see the keys_sample.json file for proper formatting

In [141]:
#this dict defines the origin.  It can have a train station, bus stop, or both.  It can also have multiple of each.
#the example implementation is Ashland & Lake.
#It will calculate transfers from the Green and Pink lines to other L lines;
#transfers from the 9 and X9 to the Orange and Red Lines;
#and transfers from the 9 and X9 to the 66 Chicago bus.
origin = {"trainstop":["40170"],
         "busstop":["14783","6035"]}

transferpoints = []
# transferpoints.append({"name":"State & Lake",
#                        "trainstop":["40260","41660"],
#                         "busstop":[]})
#this is the transfer at State & Lake.  The L stop and the Red Line stop are separate in the CTA's tracking so both are included.

transferpoints.append({"name":"Ashland Orange Line",
                       "trainstop":["41060"],
                        "busstop":["4164","14476","4163"]})
#this is the transfer between the southbound 9 and X9 and the Orange Line at Ashland

transferpoints.append({"name":"Ashland & Chicago",
                       "trainstop":[],
                        "busstop":["15843","622","556","15842"]})
#this it the transfer between the 9 and X9 and the 66

transferroutes = ["66","P","Red","Org"]

def fetchCTAtrains(trainstops):
    if trainstops == "":
        return
    train_url = "http://lapi.transitchicago.com/api/1.0/ttarrivals.aspx?key="+trainkey+"&mapid="+trainstops+"&outputType=JSON"
    #print(train_url)
    with urllib.request.urlopen(train_url) as url:
        train_info = json.loads(url.read().decode())
    return train_info

def fetchCTAbuses(busstops):
    if busstops == "":
        return
    busurl = "http://www.ctabustracker.com/bustime/api/v2/getpredictions?key="+buskey+"&stpid="+busstops+"&format=json"
    #print(busurl)
    with urllib.request.urlopen(busurl) as url:
        bus_info = json.loads(url.read().decode())
    return bus_info

def parseCTAtime(timestamp):
    return datetime.datetime.strptime(timestamp,"%Y-%m-%dT%H:%M:%S")
    #turns the CTA's timestamp into timezone naive datetime objects

In [102]:
def combineAPI(start,connect):
    alltrains = ""
    for station in start["trainstop"]:
        alltrains += station+","
    alltrains = alltrains[0:-1]
    for stop in connect:
        for station in stop["trainstop"]:
            alltrains+=","+station
    allbuses = ""
    for station in start["busstop"]:
        allbuses += station+","
    allbuses = allbuses[0:-1]
    for stop in connect:
        for station in stop["busstop"]:
            allbuses+=","+station
    return alltrains,allbuses

In [118]:
def transfercalc(origin,transferpoints,traindata,busdata):
    trains = traindata["ctatt"]["eta"]
    origintrains,originbuses = [],[]
    origintrainruns,originbusruns = [],[]
    transfertrains,transferbuses = [],[]
    for train in trains:
        ETA = parseCTAtime(train["arrT"]) - parseCTAtime(train["prdt"])
        minutes_prelim = str(ETA)
        train["ETA"] = int(minutes_prelim[2:4])
        if train["staId"] in origin["trainstop"]:
            origintrains.append(train)
        else:
            transfertrains.append(train)
    buses = busdata["bustime-response"]["prd"]
    for bus in buses:
        if bus["prdctdn"] == "DUE":
            bus["prdctdn"] = 0
        if bus["stpid"] in origin["busstop"]:
            originbuses.append(bus)
        else:
            transferbuses.append(bus)
    
    for train in origintrains:
        for train2 in transfertrains:
            if train["rn"] == train2["rn"]:
                transferdict = {}
                transferdict["ETA"] = train2["ETA"]
                transfers = []
                for connector in transferpoints:
                    if train2["staId"] in connector["trainstop"]:
                        transferIDs = connector["trainstop"] + connector["busstop"]
                for train3 in transfertrains:
                    if train3["staId"] in transferIDs and train3["ETA"]>=train2["ETA"] and train3["rn"] != train2["rn"]:
                        transfers.append(train3)
                for bus in transferbuses:
                    if bus["stpid"] in transferIDs and int(bus["prdctdn"])>=train2["ETA"]:
                        transfers.append(bus)
                transferdict["connections"] = transfers
                train["transfer"] = transferdict
    
    for bus in originbuses:
        for bus2 in transferbuses:
            if bus["vid"] == bus2["vid"]:
                transferdict = {}
                transferdict["ETA"] = int(bus2["prdctdn"])
                transfers = []
                for connector in transferpoints:
                    if train2["staId"] in connector["trainstop"]:
                        transferIDs = connector["trainstop"] + connector["busstop"]
                for train in transfertrains:
                    if train["staId"] in transferIDs and train["ETA"]>=transferdict["ETA"]:
                        transfers.append(train)
                for bus3 in transferbuses:
                    if bus3["stpid"] in transferIDs and int(bus3["prdctdn"])>=transferdict["ETA"] and bus3["vid"]!=bus2["vid"]:
                        transfers.append(bus3)
                transferdict["connections"] = transfers
                bus["transfer"] = transferdict
    
    return origintrains,originbuses

In [139]:
def parseconnections(origintrains,originbuses,transferpoints,transferroutes):
    trainoutputs = []
    for train in origintrains:
        output = train["rt"]+" line to "+train["destNm"]+", "+str(train["ETA"])+" minutes"
        xfers = []
        if "transfer" in train.keys():
            for x in train["transfer"]["connections"]:
                if x["rt"] in transferroutes:
                    if "rn" in x.keys():
                        xferoutput = x["rt"]+" connection at "+x["staNm"]+" to "+x["destNm"]+", layover "+str(x["ETA"]-train["ETA"])+" minutes"
                    elif "vid" in x.keys():
                        xferoutput = x["rt"]+" connection at "+x["stpnm"]+" to "+x["des"]+", layover "+str(int(x["prdctdn"])-train["ETA"])+" minutes"
                    xfers.append(xferoutput)
        trainoutputs.append([output,xfers])
    
    busoutputs = []
    for bus in originbuses:
        output = bus["rt"]+" to "+bus["des"]+", "+bus["prdctdn"]+" minutes"
        xfers = []
        if "transfer" in bus.keys():
            for x in bus["transfer"]["connections"]:
                if x["rt"] in transferroutes:
                    if "rn" in x.keys():
                        xferoutput = x["rt"]+" connection at "+x["staNm"]+" to "+x["destNm"]+", layover "+str(x["ETA"]-int(bus["prdctdn"]))+" minutes"
                    elif "vid" in x.keys():
                        xferoutput = x["rt"]+" connection at "+x["stpnm"]+" to "+x["des"]+", layover "+str(x["prdctdn"]-int(bus["prdctdn"]))+" minutes"
                    xfers.append(xferoutput)
        busoutputs.append([output,xfers])
    
    return trainoutputs,busoutputs

In [142]:
stopIDs = combineAPI(origin,transferpoints)
traindata = fetchCTAtrains(stopIDs[0])
busdata = fetchCTAbuses(stopIDs[1])
xferdata = transfercalc(origin,transferpoints,traindata,busdata)
parseconnections(xferdata[0],xferdata[1],transferpoints,transferroutes)

([['Pink line to 54th/Cermak, 2 minutes', []],
  ['Pink line to Loop, 2 minutes', []],
  ['G line to Ashland/63rd, 3 minutes', []],
  ['G line to Harlem/Lake, 10 minutes', []],
  ['G line to Cottage Grove, 13 minutes', []],
  ['Pink line to 54th/Cermak, 14 minutes', []],
  ['G line to Harlem/Lake, 15 minutes', []],
  ['G line to Harlem/Lake, 18 minutes', []],
  ['Pink line to Loop, 20 minutes', []],
  ['Pink line to Loop, 29 minutes', []],
  ['Pink line to Loop, 37 minutes', []],
  ['Pink line to Loop, 46 minutes', []],
  ['Pink line to Loop, 54 minutes', []]],
 [['X9 to Irving Park/Broadway, 2 minutes',
   ['Org connection at Ashland to Midway, layover 8 minutes',
    'Org connection at Ashland to Loop, layover 9 minutes']],
  ['X9 to 95th, 6 minutes',
   ['Org connection at Ashland to Loop, layover -5 minutes',
    'Org connection at Ashland to Midway, layover -4 minutes',
    'Org connection at Ashland to Midway, layover 4 minutes',
    'Org connection at Ashland to Loop, layover 5 